In [1]:
import pandas as pd
import numpy as np
import math
from datetime import datetime
import dask.dataframe as dd
import dask.array as da
import time
import statsmodels.api as sm
import statistics

In [26]:
return_daily = pd.read_csv("question5.csv")#2.55GB

In [27]:
return_daily

,date,COMNAM,RET,day,month,year
0,1996-01-02,ENERGY WEST INC,-0.026667,2,1,1996
1,1996-01-03,ENERGY WEST INC,0.041096,3,1,1996
2,1996-01-04,ENERGY WEST INC,-0.039474,4,1,1996
3,1996-01-05,ENERGY WEST INC,-0.041096,5,1,1996
4,1996-01-08,ENERGY WEST INC,0.000000,8,1,1996
...,...,...,...,...,...,...
51981194,2023-06-23,TESLA INC,-0.030271,23,6,2023
51981195,2023-06-26,TESLA INC,-0.060600,26,6,2023
51981196,2023-06-27,TESLA INC,0.038000,27,6,2023
51981197,2023-06-28,TESLA INC,0.024100,28,6,2023


In [28]:
list_total = list(return_daily['COMNAM'].unique())
#list_new = ['ENERGY WEST INC','ENERGY INC']

In [9]:
#beacause of the huge size of this return_daily file, 
#I split the data into 10 parts with nearly 6 millions pieces of data per file. 
#In each file, there are nearlly 2700 kinds of firm stocks
testt = return_daily.head(5731970) 

In [18]:
testt #Now I use the first 2700 firm stocks as test.

,date,COMNAM,RET,day,month,year
0,1996-01-02,ENERGY WEST INC,-0.026667,2,1,1996
1,1996-01-03,ENERGY WEST INC,0.041096,3,1,1996
2,1996-01-04,ENERGY WEST INC,-0.039474,4,1,1996
3,1996-01-05,ENERGY WEST INC,-0.041096,5,1,1996
4,1996-01-08,ENERGY WEST INC,0.000000,8,1,1996
...,...,...,...,...,...,...
5731965,2016-06-06,OPOWER INC,-0.000972,6,6,2016
5731966,2016-06-07,OPOWER INC,0.000973,7,6,2016
5731967,2016-06-08,OPOWER INC,0.000000,8,6,2016
5731968,2016-06-09,OPOWER INC,0.000000,9,6,2016


In [29]:
list_new = list_total[:2731] 
#the list_new records the the first 2700 stock names(comnam) in order.
#in the next code block, I delete the stocks will month observations less than 17.

In [19]:
start = time.time()
ini_index = last_index = 0 #in the first round, the index should be 0.
for ele in list_new:  
    for in_dex, row in testt.iloc[ini_index:].iterrows():
        if row['COMNAM'] != ele:
            last_index = in_dex
            break
    if ini_index == last_index:
        last_index = testt.index[-1]
    test_stock = testt.iloc[ini_index:last_index+1]
    for j in list(test_stock['year'].unique()):  
            a = test_stock[test_stock['year'] == j]
            for i in list(a['month'].unique()):    
                if len(a[a['month'] == i]) < 17:
                    #print(len(a[a['month'] == i]))
                    testt = testt.drop(a[a['month'] == i].index, axis = 0)
    ini_index = last_index

end = time.time()
print(end - start)

793.0875704288483


In [22]:
testt.to_csv("1st_list.csv", index=False)

In [2]:
factors = pd.read_csv('daily_factors.csv')
factors = factors.drop(labels = 'datee', axis=1)
factors

,Mkt-RF,SMB,HML,RF,Date
0,0.59,-0.31,-0.21,0.019,1996-01-02
1,-0.08,-0.26,0.50,0.019,1996-01-03
2,-0.82,-0.38,0.22,0.019,1996-01-04
3,-0.15,0.69,-0.12,0.019,1996-01-05
4,0.24,-0.20,-0.11,0.019,1996-01-08
...,...,...,...,...,...
6937,0.25,-0.04,-0.79,0.022,2023-07-25
6938,0.02,0.68,1.03,0.022,2023-07-26
6939,-0.74,-0.90,0.27,0.022,2023-07-27
6940,1.14,0.53,-0.33,0.022,2023-07-28


In [3]:
list1 = pd.read_csv("1st_list.csv")
list1

,date,COMNAM,RET,day,month,year
0,1996-01-02,ENERGY WEST INC,-0.026667,2,1,1996
1,1996-01-03,ENERGY WEST INC,0.041096,3,1,1996
2,1996-01-04,ENERGY WEST INC,-0.039474,4,1,1996
3,1996-01-05,ENERGY WEST INC,-0.041096,5,1,1996
4,1996-01-08,ENERGY WEST INC,0.000000,8,1,1996
...,...,...,...,...,...,...
5721817,2016-06-06,OPOWER INC,-0.000972,6,6,2016
5721818,2016-06-07,OPOWER INC,0.000973,7,6,2016
5721819,2016-06-08,OPOWER INC,0.000000,8,6,2016
5721820,2016-06-09,OPOWER INC,0.000000,9,6,2016


In [4]:
link_stock = list1.drop(labels=['day'], axis=1)
link_stock

,date,COMNAM,RET,month,year
0,1996-01-02,ENERGY WEST INC,-0.026667,1,1996
1,1996-01-03,ENERGY WEST INC,0.041096,1,1996
2,1996-01-04,ENERGY WEST INC,-0.039474,1,1996
3,1996-01-05,ENERGY WEST INC,-0.041096,1,1996
4,1996-01-08,ENERGY WEST INC,0.000000,1,1996
...,...,...,...,...,...
5721817,2016-06-06,OPOWER INC,-0.000972,6,2016
5721818,2016-06-07,OPOWER INC,0.000973,6,2016
5721819,2016-06-08,OPOWER INC,0.000000,6,2016
5721820,2016-06-09,OPOWER INC,0.000000,6,2016


In [19]:
after_merge = pd.merge_ordered(link_stock, factors, left_on = 'date', right_on = 'Date', left_by = 'COMNAM', how = 'left')
#z = pd.merge_ordered(x, y, on='state')

In [20]:
after_merge

,date,COMNAM,RET,month,year,Mkt-RF,SMB,HML,RF,Date
0,1996-01-02,ENERGY WEST INC,-0.026667,1,1996,0.59,-0.31,-0.21,0.019,1996-01-02
1,1996-01-03,ENERGY WEST INC,0.041096,1,1996,-0.08,-0.26,0.50,0.019,1996-01-03
2,1996-01-04,ENERGY WEST INC,-0.039474,1,1996,-0.82,-0.38,0.22,0.019,1996-01-04
3,1996-01-05,ENERGY WEST INC,-0.041096,1,1996,-0.15,0.69,-0.12,0.019,1996-01-05
4,1996-01-08,ENERGY WEST INC,0.000000,1,1996,0.24,-0.20,-0.11,0.019,1996-01-08
...,...,...,...,...,...,...,...,...,...,...
5721817,2016-06-06,OPOWER INC,-0.000972,6,2016,0.60,0.67,0.31,0.001,2016-06-06
5721818,2016-06-07,OPOWER INC,0.000973,6,2016,0.12,0.16,0.10,0.001,2016-06-07
5721819,2016-06-08,OPOWER INC,0.000000,6,2016,0.37,0.45,-0.09,0.001,2016-06-08
5721820,2016-06-09,OPOWER INC,0.000000,6,2016,-0.27,-0.40,-0.24,0.001,2016-06-09


In [21]:
new_after = after_merge
new_after['excess_return'] = new_after['RET'] - new_after['RF']
new_after = new_after.drop(labels=['RET', 'RF', 'Date'], axis=1)
new_after

,date,COMNAM,month,year,Mkt-RF,SMB,HML,excess_return
0,1996-01-02,ENERGY WEST INC,1,1996,0.59,-0.31,-0.21,-0.045667
1,1996-01-03,ENERGY WEST INC,1,1996,-0.08,-0.26,0.50,0.022096
2,1996-01-04,ENERGY WEST INC,1,1996,-0.82,-0.38,0.22,-0.058474
3,1996-01-05,ENERGY WEST INC,1,1996,-0.15,0.69,-0.12,-0.060096
4,1996-01-08,ENERGY WEST INC,1,1996,0.24,-0.20,-0.11,-0.019000
...,...,...,...,...,...,...,...,...
5721817,2016-06-06,OPOWER INC,6,2016,0.60,0.67,0.31,-0.001972
5721818,2016-06-07,OPOWER INC,6,2016,0.12,0.16,0.10,-0.000027
5721819,2016-06-08,OPOWER INC,6,2016,0.37,0.45,-0.09,-0.001000
5721820,2016-06-09,OPOWER INC,6,2016,-0.27,-0.40,-0.24,-0.001000


In [23]:
new_after.to_csv('link_list1.csv', index=False)

In [24]:
new_afterlink = pd.read_csv('link_list1.csv')
new_afterlink

,date,COMNAM,month,year,Mkt-RF,SMB,HML,excess_return
0,1996-01-02,ENERGY WEST INC,1,1996,0.59,-0.31,-0.21,-0.045667
1,1996-01-03,ENERGY WEST INC,1,1996,-0.08,-0.26,0.50,0.022096
2,1996-01-04,ENERGY WEST INC,1,1996,-0.82,-0.38,0.22,-0.058474
3,1996-01-05,ENERGY WEST INC,1,1996,-0.15,0.69,-0.12,-0.060096
4,1996-01-08,ENERGY WEST INC,1,1996,0.24,-0.20,-0.11,-0.019000
...,...,...,...,...,...,...,...,...
5721817,2016-06-06,OPOWER INC,6,2016,0.60,0.67,0.31,-0.001972
5721818,2016-06-07,OPOWER INC,6,2016,0.12,0.16,0.10,-0.000027
5721819,2016-06-08,OPOWER INC,6,2016,0.37,0.45,-0.09,-0.001000
5721820,2016-06-09,OPOWER INC,6,2016,-0.27,-0.40,-0.24,-0.001000


In [32]:
new_afterlink['IVOL'] = None
new_afterlink

,date,COMNAM,month,year,Mkt-RF,SMB,HML,excess_return,IVOL
0,1996-01-02,ENERGY WEST INC,1,1996,0.59,-0.31,-0.21,-0.045667,None
1,1996-01-03,ENERGY WEST INC,1,1996,-0.08,-0.26,0.50,0.022096,None
2,1996-01-04,ENERGY WEST INC,1,1996,-0.82,-0.38,0.22,-0.058474,None
3,1996-01-05,ENERGY WEST INC,1,1996,-0.15,0.69,-0.12,-0.060096,None
4,1996-01-08,ENERGY WEST INC,1,1996,0.24,-0.20,-0.11,-0.019000,None
...,...,...,...,...,...,...,...,...,...
5721817,2016-06-06,OPOWER INC,6,2016,0.60,0.67,0.31,-0.001972,None
5721818,2016-06-07,OPOWER INC,6,2016,0.12,0.16,0.10,-0.000027,None
5721819,2016-06-08,OPOWER INC,6,2016,0.37,0.45,-0.09,-0.001000,None
5721820,2016-06-09,OPOWER INC,6,2016,-0.27,-0.40,-0.24,-0.001000,None
